In [2]:
import pandas as pd

df= pd.read_csv('data/test/testn225.csv')
df2= pd.read_csv('data/test/multi_log_reg/multi_n225.csv')

In [3]:
df2.head()

,Date,change,mean,std,effect
0,2008-08-01,-1.370689,0.439156,1.641390,1
1,2008-08-04,-1.147270,-1.053739,0.968103,0
2,2008-08-05,-0.326847,1.242894,2.023114,0
3,2008-08-06,1.496696,0.206239,0.663693,1
4,2008-08-07,-1.003169,-0.430029,1.064390,0


In [ ]:
df2 = df2.rename(columns={'Date': 'date'})
df2['date'] = pd.to_datetime(df2['date']).dt.tz_localize('Japan')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
df2['date'] = pd.to_datetime(df2['date']).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
df2_subset = df2[['date', 'effect']]

# Merge df and df2_subset on the Date column
merged_df = pd.merge(df, df2_subset, on='date', how='left')

merged_df.head()


TypeError: Already tz-aware, use tz_convert to convert.

In [11]:
merged_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,tsunami,rev_dist,effect
0,0,5,5,^N225,2008-08-01T00:00:00+0900,-181.980469,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,1
1,1,17,17,^N225,2008-08-04T00:00:00+0900,-150.100586,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0
2,2,29,29,^N225,2008-08-05T00:00:00+0900,-42.349609,1,6.3,6.3,6.3,4818.209478,4818.209478,0.0,0.000208,0
3,3,41,41,^N225,2008-08-06T00:00:00+0900,195.459961,1,6.0,6.0,6.0,3159.157480,3159.157480,0.0,0.000317,1
4,4,53,53,^N225,2008-08-07T00:00:00+0900,-133.000000,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0


In [20]:
merged_df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1','Unnamed: 0'], inplace=True)
merged_df.head()

,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,tsunami,rev_dist,effect
0,^N225,2008-08-01T00:00:00+0900,-181.980469,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,1
1,^N225,2008-08-04T00:00:00+0900,-150.100586,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0
2,^N225,2008-08-05T00:00:00+0900,-42.349609,1,6.3,6.3,6.3,4818.209478,4818.209478,0.0,0.000208,0
3,^N225,2008-08-06T00:00:00+0900,195.459961,1,6.0,6.0,6.0,3159.157480,3159.157480,0.0,0.000317,1
4,^N225,2008-08-07T00:00:00+0900,-133.000000,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0


In [21]:
merged_df['effect'].unique()

array([1, 0, 2, 3])

In [22]:
import statsmodels.api as sm

# Add a constant to the independent variables (for the intercept term)
X = sm.add_constant(merged_df[['num', 'max', 'min_dist','tsunami']])
y = merged_df['effect']

# Fit the logistic regression model
mlogit_model = sm.MNLogit(y, X)
result = mlogit_model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.958706
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                 effect   No. Observations:                 3664
Model:                        MNLogit   Df Residuals:                     3649
Method:                           MLE   Df Model:                           12
Date:                Tue, 18 Feb 2025   Pseudo R-squ.:                0.001686
Time:                        11:45:17   Log-Likelihood:                -3512.7
converged:                       True   LL-Null:                       -3518.6
Covariance Type:            nonrobust   LLR p-value:                    0.4567
  effect=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9458      0.049    -19.265      0.000      -1.042      -0.850
num           -0.0450      0.